This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' # Key removed after assignment was completed. 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests as re
import json
from collections import defaultdict

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = re.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2019-04-09&end_date=2019-04-09&api_key=' + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-04-09T22:31:49.414Z',
  'newest_available_date': '2019-04-09',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-04-09',
  'end_date': '2019-04-09',
  'data': [['2019-04-09',
    None,
    75.8,
    73.3,
    75.3,
    None,
    183859.0,
    13789864.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Analysis: 

In [5]:
# Use the API to get a JSON object containing all AFX_X data for 2017. 
data_json = re.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

In [6]:
data_dict = data_json.json() # Convert JSON object into a dictionary.
columns = data_dict['dataset']['column_names'] # Isolate the columns of the data into their own list. 
values = data_dict['dataset']['data'] # Isolate the actual values from the dataset and store as a 2-D list. 
for index, column in enumerate(columns):
    print(index, column) # Print the names of the columns next to their associated index for easy reference. 

0 Date
1 Open
2 High
3 Low
4 Close
5 Change
6 Traded Volume
7 Turnover
8 Last Price of the Day
9 Daily Traded Units
10 Daily Turnover


In [7]:
# Determine the min and max opening prices.
opening_prices = [row[1] for row in values if type(row[1]) == float]
max_opening = sorted(opening_prices)[-1]
min_opening = sorted(opening_prices)[0]

In [8]:
# Determine the maximum daily change (according to high and low price).
daily_changes = [row[2] - row[3] for row in values if type(row[2]) == float and type(row[3]) == float]
max_daily_change = max(daily_changes)

In [9]:
# Determine the maximum change between any two days (according to closing price). 
closing_prices = [row[4] for row in values]
closing_pairs = zip(closing_prices, closing_prices[1:]) # Creates a list of tuples pairing adjacent closing prices.
two_day_changes = [day2 - day1 for (day1, day2) in closing_pairs if type(day1) == float and type(day2) == float] 
max_two_day_change = max(two_day_changes)

In [10]:
# Determine the average traded volume.
traded_volumes = [row[6] for row in values if type(row[6]) == float] 
avg_traded_volume = sum(traded_volumes)/len(traded_volumes)

In [11]:
# find_median() function takes a list as input and returns the median of that list. 
def find_median(a_list):
    l = sorted(a_list)
    n = len(a_list)
    if n % 2 == 0: # If list has an odd number of elements, set median equal to the average of the middle two elements.
        median = (l[int(n/2)] + l[int(n/2) - 1]) / 2
    else: # If the list has an even number of elements, set the median equal to the middle element. 
        median = l[int(n/2)]
    return median

median_traded_volume = find_median(traded_volumes) # Determine median traded volume. 

# Results:
The statistics below correspond to stock data for the Carl Zeiss Meditec company during 2017.    

In [12]:
print("The minimum opening price was: $%.2f" % min_opening)
print("The maximum opening price was: $%.2f" % max_opening)
print("The maximum daily change in stock price (based on high and low values) was: $%.2f" % max_daily_change)
print("The maximum change in stock price between two days (based on closing values) was: $%.2f" % max_two_day_change)
print("The average daily traded stock volume was: $%.2f" % avg_traded_volume)
print("The median daily traded stock volume was: $%.2f" % median_traded_volume)

The minimum opening price was: $34.00
The maximum opening price was: $53.11
The maximum daily change in stock price (based on high and low values) was: $2.81
The maximum change in stock price between two days (based on closing values) was: $2.56
The average daily traded stock volume was: $89124.34
The median daily traded stock volume was: $76286.00
